# Let's try MySQLdb
Before we start MySQL in python, make sure you have installed MySQL, and turned on the MySQL server instance. To install mysql, Simply open up your terminal and type `sudo install mysql-server` , you may need to enter administration password to finish the installation. Next, type `sudo mysql -u root` to log on to MySQL. Once you have logged in, you may create a database, user and a password to authorise interaction with the database, the instructions are shown below: 
1. `CREATE DATABASE sql_master;`
2. `USE sql_master;` 
3. `CREATE USER 'sec_user'@'localhost' IDENTIFIED BY '9008';`
4. `GRANT ALL PRIVILEGES ON sql_master.* TO 'sec_user'@'localhost';`
5. `FLUSH PRIVILEGES;`

The above steps authorise `sec_user` to interact with the database. Next, we will learn how to create a new table, insert data into our table, as well as updating our data.   

## Create a new table in MySQL Database

In [41]:
import MySQLdb as mdb

# Obtain a database connection to the MySQL instance
db_host = 'localhost'
db_user = 'sec_user'
db_pass = '9008'
db_name = 'sql_master'
con = mdb.connect(db_host, db_user, db_pass, db_name)


sql = """CREATE TABLE IF NOT EXISTS daily_google_price( 
    price_date datetime NOT NULL, 
    open_price decimal(19,4) NULL, 
    high_price decimal(19,4) NULL, 
    low_price decimal(19,4) NULL, 
    close_price decimal(19,4) NULL, 
    adjusted_close_price decimal(19,4) NULL, 
    volume bigint NULL)"""
# create a new table named daily_google_price if it does not exist  
# decimal(19,4) means 19 digits and 4 decimal places, this avoids rounding error
# bigint allows large number such as trading volume 

with con: 
    cur = con.cursor()
    cur.execute(sql)
    con.commit()
    print("Table Created!!!")

Table Created!!!


## Download and pre-process financial data 

In [42]:
import datetime
import pandas_datareader.data as data

# download data from Yahoo Finance
start = datetime.datetime(2016, 1, 1)
end = datetime.datetime(2020, 12, 31)
google_data = data.DataReader("GOOG", "yahoo", start, end)
print("raw data:", google_data)

# create a 7-tuple containing the datetime, high price, low price, open price, close price, volume, and adj price 
prices = []
for y in range(len(google_data)): 
    prices.append((google_data.index[y], google_data["High"][y], google_data["Low"][y], google_data["Open"][y], 
                  google_data["Close"][y], google_data["Volume"][y], google_data["Adj Close"][y]))
print("Preprocessed data:",prices[0:10])

raw data:                    High          Low         Open        Close   Volume  \
Date                                                                      
2015-12-31   769.500000   758.340027   769.500000   758.880005  1500900   
2016-01-04   744.059998   731.257996   743.000000   741.840027  3272800   
2016-01-05   752.000000   738.640015   746.450012   742.580017  1950700   
2016-01-06   747.179993   728.919983   730.000000   743.619995  1947000   
2016-01-07   738.500000   719.059998   730.309998   726.390015  2963700   
...                 ...          ...          ...          ...      ...   
2020-12-24  1746.000000  1729.109985  1735.000000  1738.849976   346800   
2020-12-28  1790.728027  1746.334961  1751.635010  1776.089966  1393000   
2020-12-29  1792.439941  1756.089966  1787.790039  1758.719971  1299400   
2020-12-30  1765.094971  1725.599976  1762.010010  1739.520020  1306100   
2020-12-31  1758.930054  1735.420044  1735.420044  1751.880005  1011900   

              

## Insert data into mysql table

In [43]:
import pandas as pd

count = 0
for data in prices:
    daily_data = data[0], data[1], data[2], data[3], data[4], data[6], data[5]
    
    sql = """INSERT INTO daily_google_price( price_date, open_price, high_price, low_price, 
        close_price, adjusted_close_price, volume) 
        VALUES ('%s', '%s', '%s', '%s', '%s', '%s', '%s')""" %(daily_data) 
    
    # Obtain a database connection to the MySQL instance
    db_host = 'localhost'
    db_user = 'sec_user'
    db_pass = '9008'
    db_name = 'sql_master'
    con = mdb.connect(db_host, db_user, db_pass, db_name)
 
    with con: 
        cur = con.cursor()
        cur.execute(sql)
        con.commit()
        count += 1
        
        # print the table when it is done inserting 
        if count == len(prices):
            sql2 = "SELECT * FROM daily_google_price"
            googling = pd.read_sql_query(sql2, con=con)#, index_col= 'price_date')
            print(googling) 
            
print("data saved in database!")

     price_date  open_price  high_price  low_price  close_price  \
0    2015-12-31    769.5000     758.340    769.500       758.88   
1    2016-01-04    744.0600     731.258    743.000       741.84   
2    2016-01-05    752.0000     738.640    746.450       742.58   
3    2016-01-06    747.1800     728.920    730.000       743.62   
4    2016-01-07    738.5000     719.060    730.310       726.39   
...         ...         ...         ...        ...          ...   
1255 2020-12-24   1746.0000    1729.110   1735.000      1738.85   
1256 2020-12-28   1790.7280    1746.335   1751.635      1776.09   
1257 2020-12-29   1792.4399    1756.090   1787.790      1758.72   
1258 2020-12-30   1765.0950    1725.600   1762.010      1739.52   
1259 2020-12-31   1758.9301    1735.420   1735.420      1751.88   

      adjusted_close_price   volume  
0                   758.88  1500900  
1                   741.84  3272800  
2                   742.58  1950700  
3                   743.62  1947000  
4    

## Select data by Sorting in Descending Order 

In [54]:
# Obtain a database connection to the MySQL instance
db_host = 'localhost'
db_user = 'sec_user'
db_pass = '9008'
db_name = 'sql_master'
con = mdb.connect(db_host, db_user, db_pass, db_name)
 
with con: 
    cur = con.cursor()
    sql2 = "SELECT * FROM daily_google_price ORDER BY open_price DESC"
    cur.execute(sql2)
    googling = pd.read_sql_query(sql2, con=con)#, index_col= 'price_date')
    print(googling) 

     price_date  open_price  high_price  low_price  close_price  \
0    2020-12-03    1847.200   1822.6500    1824.01      1826.77   
1    2020-12-02    1835.650   1789.4730    1798.10      1827.95   
2    2020-12-09    1834.270   1767.8101    1812.01      1784.13   
3    2020-12-04    1833.160   1816.9900    1824.52      1827.99   
4    2020-12-07    1832.370   1805.7800    1819.00      1819.48   
...         ...         ...         ...        ...          ...   
1255 2016-02-11     689.350    668.8680     675.00       683.11   
1256 2016-06-29     687.429    681.4100     683.00       684.11   
1257 2016-02-08     684.030    663.0600     667.85       682.74   
1258 2016-06-28     680.330    673.0000     678.97       680.04   
1259 2016-06-27     672.300    663.2840     671.00       668.26   

      adjusted_close_price   volume  
0                  1826.77  1227300  
1                  1827.95  1222000  
2                  1784.13  1507600  
3                  1827.99  1378200  
4    

## Select data by position 

In [57]:
# Obtain a database connection to the MySQL instance
db_host = 'localhost'
db_user = 'sec_user'
db_pass = '9008'
db_name = 'sql_master'
con = mdb.connect(db_host, db_user, db_pass, db_name)
 
with con: 
    cur = con.cursor()
    # Select the largest second number on open price
    sql2 = "SELECT * FROM daily_google_price ORDER BY open_price DESC LIMIT 1 OFFSET 2"
    cur.execute(sql2)
    #print(cur.fetchmany(size=2))
    googling = pd.read_sql_query(sql2, con=con)#, index_col= 'price_date')
    print(googling) 

  price_date  open_price  high_price  low_price  close_price  \
0 2020-12-09     1834.27   1767.8101    1812.01      1784.13   

   adjusted_close_price   volume  
0               1784.13  1507600  


## Select data rows that have similar column value

In [64]:
# Obtain a database connection to the MySQL instance
db_host = 'localhost'
db_user = 'sec_user'
db_pass = '9008'
db_name = 'sql_master'
con = mdb.connect(db_host, db_user, db_pass, db_name)
 
with con: 
    cur = con.cursor()
    # Check the open prices that have multiple rows in our data
    # in other words, multiple days that have the same open price
    print("Open prices that have multiple rows:")
    sql2 = "SELECT * FROM daily_google_price GROUP BY open_price HAVING count(open_price)>1"
    cur.execute(sql2)
    googling = pd.read_sql_query(sql2, con=con)#, index_col= 'price_date')
    print(googling)
    print("\n")
    
    cur = con.cursor()
    # verify the open price (741.69) to find out how many rows have the same price 
    print("Rows that have the same 741.69 open price:")
    sql = "SELECT * FROM daily_google_price WHERE open_price=741.6900"
    cur.execute(sql)
    googling = pd.read_sql_query(sql, con=con)#, index_col= 'price_date')
    print(googling)

Open prices that have multiple rows:
   price_date  open_price  high_price  low_price  close_price  \
0  2016-07-21    741.6900    735.8310   740.3600     738.6300   
1  2016-03-22    745.0000    737.4600   737.4600     740.7500   
2  2016-02-03    774.5000    720.5000   770.2200     726.9500   
3  2017-02-14    823.0000    816.0000   819.0000     820.4500   
4  2017-10-30   1024.9700   1007.5000  1014.0000    1017.1100   
5  2017-12-27   1058.3700   1048.0500  1057.3900    1049.3700   
6  2018-06-07   1135.8199   1116.5200  1131.3199    1123.8600   
7  2018-01-26   1175.8400   1158.1100  1175.0800    1175.8400   
8  2018-09-12   1178.6100   1158.3600  1172.7200    1162.8199   
9  2018-01-29   1186.8900   1171.9800  1176.4800    1175.5800   
10 2019-08-20   1196.0601   1182.1100  1195.2500    1182.6899   
11 2018-07-19   1200.0000   1183.3199  1191.0000    1186.9600   
12 2018-08-16   1226.0000   1202.5500  1224.7300    1206.4900   
13 2018-08-03   1230.0000   1215.0601  1229.6200    1

## Delete row that satisfied the condition

In [71]:
# Obtain a database connection to the MySQL instance
db_host = 'localhost'
db_user = 'sec_user'
db_pass = '9008'
db_name = 'sql_master'
con = mdb.connect(db_host, db_user, db_pass, db_name)
    
sql = """DELETE FROM daily_google_price WHERE volume>1500000"""
with con:
    cur = con.cursor()
    cur.execute(sql)
    con.commit()
    sql2 = "SELECT * FROM daily_google_price"
    googling = pd.read_sql_query(sql2, con=con, index_col= 'price_date')
    print(googling)
    


            open_price  high_price  low_price  close_price  \
price_date                                                   
2016-01-26    718.2800     706.480    713.850       713.04   
2016-03-09    705.6800     694.000    698.470       705.24   
2016-03-22    745.0000     737.460    737.460       740.75   
2016-03-23    745.7200     736.150    742.360       738.06   
2016-03-28    738.9900     732.500    736.790       733.53   
...                ...         ...        ...          ...   
2020-12-24   1746.0000    1729.110   1735.000      1738.85   
2020-12-28   1790.7280    1746.335   1751.635      1776.09   
2020-12-29   1792.4399    1756.090   1787.790      1758.72   
2020-12-30   1765.0950    1725.600   1762.010      1739.52   
2020-12-31   1758.9301    1735.420   1735.420      1751.88   

            adjusted_close_price   volume  
price_date                                 
2016-01-26                713.04  1331700  
2016-03-09                705.24  1421500  
2016-03-22       

**Note:** Notice from the output above, when we specify delete rows with volume > 1500000 , we deleted nearly half of the data, from 1260 rows to 663 rows.  

## Add new column to table

In [65]:
# Obtain a database connection to the MySQL instance
db_host = 'localhost'
db_user = 'sec_user'
db_pass = '9008'
db_name = 'sql_master'
con = mdb.connect(db_host, db_user, db_pass, db_name)
    
sql = """ALTER TABLE daily_google_price ADD Exchange Varchar(255) """
sql2 = """UPDATE daily_google_price SET Exchange = 'NASDAQ' """

cur = con.cursor()
cur.execute(sql)
cur.execute(sql2)
con.commit()

googling = pd.read_sql_query("SELECT * FROM daily_google_price", con=con, index_col= 'price_date')
print(googling)

            open_price  high_price  low_price  close_price  \
price_date                                                   
2015-12-31    769.5000     758.340    769.500       758.88   
2016-01-04    744.0600     731.258    743.000       741.84   
2016-01-05    752.0000     738.640    746.450       742.58   
2016-01-06    747.1800     728.920    730.000       743.62   
2016-01-07    738.5000     719.060    730.310       726.39   
...                ...         ...        ...          ...   
2020-12-24   1746.0000    1729.110   1735.000      1738.85   
2020-12-28   1790.7280    1746.335   1751.635      1776.09   
2020-12-29   1792.4399    1756.090   1787.790      1758.72   
2020-12-30   1765.0950    1725.600   1762.010      1739.52   
2020-12-31   1758.9301    1735.420   1735.420      1751.88   

            adjusted_close_price   volume Exchange  
price_date                                          
2015-12-31                758.88  1500900   NASDAQ  
2016-01-04                741.84  

## Delete column from mysql table

In [12]:
# Obtain a database connection to the MySQL instance
db_host = 'localhost'
db_user = 'sec_user'
db_pass = '9008'
db_name = 'sql_master'
con = mdb.connect(db_host, db_user, db_pass, db_name)
    
sql = """ALTER TABLE daily_google_price DROP Exchange"""

cur = con.cursor()
cur.execute(sql)
con.commit()

googling = pd.read_sql_query("SELECT * FROM daily_google_price", con=con, index_col= 'price_date')
print(googling)

            open_price  high_price  low_price  close_price  \
price_date                                                   
2016-01-26    718.2800     706.480    713.850       713.04   
2016-03-09    705.6800     694.000    698.470       705.24   
2016-03-22    745.0000     737.460    737.460       740.75   
2016-03-23    745.7200     736.150    742.360       738.06   
2016-03-28    738.9900     732.500    736.790       733.53   
...                ...         ...        ...          ...   
2020-12-24   1746.0000    1729.110   1735.000      1738.85   
2020-12-28   1790.7280    1746.335   1751.635      1776.09   
2020-12-29   1792.4399    1756.090   1787.790      1758.72   
2020-12-30   1765.0950    1725.600   1762.010      1739.52   
2020-12-31   1758.9301    1735.420   1735.420      1751.88   

            adjusted_close_price   volume  
price_date                                 
2016-01-26                713.04  1331700  
2016-03-09                705.24  1421500  
2016-03-22       

## Drop tables in your database

In [30]:
db_host = 'localhost'
db_user = 'sec_user'
db_pass = '9008'
db_name = 'sql_master'
con = mdb.connect(db_host, db_user, db_pass, db_name)

# See existing tables in your database
cur = con.cursor()
cur.execute("""SHOW Tables""")
print("Here's the tables we have in our database: ")
print(cur.fetchall())

cur.execute("""DROP TABLE daily_google_price""")
con.commit()
print("Drop table in progress...\n")

# Check our database table now
print("Database table: ")
cur.execute("""SHOW Tables""")
print(cur.fetchall())
print("We have dropped the daily google price table!!")

Here's the tables we have in our database: 
(('‘daily_price‘',), ('daily_google_price',))
Drop table in progress...

Database table: 
(('‘daily_price‘',),)
We have dropped the daily google price table!!


# Lets try MongoDB
Before we start connecting to MongoDB in python, make sure you have installed MongoDB, and successfully run `sudo mongod` on your terminal to accept connection from the 27017 port. Below MongoDB database tutorial for python is inspired by **https://www.tutorialspoint.com/python_data_access/**, and it is also a self-learning practice. After you are done, it is wise to run `ps wuax | grep mongo` and `sudo kill (instance number)` to terminate the connection completely. 

## Create Database and Collection, then insert data into Collection

In [33]:
import pymongo
from pymongo import MongoClient

#Creating a pymongo client
client = MongoClient('localhost', 27017)

#Getting the database instance
db = client['mongo_database']
print("Database created!")

#Verification
print("List of Mongo databases:")
print(client.list_database_names())
print("\n")

#Creating a collection 
collection = db['mongodb_table']
print("collection(mongodb_table) created!")

data = [
   {
      "id": "101", 
      "stock": "Google", 
      "price": "26", 
      "exchange": "London"
   },
   {
      "id": "102", 
      "stock": "Facebook", 
      "price": "27", 
      "exchange": "Nasdaq"
   },
   {
      "id": "103", 
      "stock": "Microsoft", 
      "price": "28", 
      "exchange": "Dow"
   }
]

res = collection.insert_many(data)
print("Data inserted!")
for record in collection.find():
    print(record)

Database created!
List of Mongo databases:
['local', 'mongo_database']


collection(mongodb_table) created!
Data inserted!
{'_id': ObjectId('612da6aa261ab319fa8517c2'), 'id': '101', 'stock': 'Google', 'price': '26', 'exchange': 'London'}
{'_id': ObjectId('612da6aa261ab319fa8517c3'), 'id': '102', 'stock': 'Facebook', 'price': '27', 'exchange': 'Nasdaq'}
{'_id': ObjectId('612da6aa261ab319fa8517c4'), 'id': '103', 'stock': 'Microsoft', 'price': '28', 'exchange': 'Dow'}


## Retrieve data from Mongo Collection

In [35]:
#Retrieving all our data 
print("all record data:")
for record in collection.find():
    print(record)
print("\n")

#Retrieving a record for Google stock using the find_one() method
print("Record for google: ")
print(collection.find_one({"exchange": "London"}))
print("\n")

# Retrieving record that has stock price greater than 26 
print("Retrieve Stock price greater than 26:")
for i in collection.find({"price":{"$gt":"26"}}):
    print(i)

all record data:
{'_id': ObjectId('612da6aa261ab319fa8517c2'), 'id': '101', 'stock': 'Google', 'price': '26', 'exchange': 'London'}
{'_id': ObjectId('612da6aa261ab319fa8517c3'), 'id': '102', 'stock': 'Facebook', 'price': '27', 'exchange': 'Nasdaq'}
{'_id': ObjectId('612da6aa261ab319fa8517c4'), 'id': '103', 'stock': 'Microsoft', 'price': '28', 'exchange': 'Dow'}


Record for google: 
{'_id': ObjectId('612da6aa261ab319fa8517c2'), 'id': '101', 'stock': 'Google', 'price': '26', 'exchange': 'London'}


Retrieve Stock price greater than 26:
{'_id': ObjectId('612da6aa261ab319fa8517c3'), 'id': '102', 'stock': 'Facebook', 'price': '27', 'exchange': 'Nasdaq'}
{'_id': ObjectId('612da6aa261ab319fa8517c4'), 'id': '103', 'stock': 'Microsoft', 'price': '28', 'exchange': 'Dow'}


## Delete rows that satisfy the condition

In [36]:
#Deleting multiple documents
collection.delete_many({"stock":"Microsoft"})
print("After deleting the row, we have:")
for i in collection.find():
    print(i)

After deleting the row, we have:
{'_id': ObjectId('612da6aa261ab319fa8517c2'), 'id': '101', 'stock': 'Google', 'price': '26', 'exchange': 'London'}
{'_id': ObjectId('612da6aa261ab319fa8517c3'), 'id': '102', 'stock': 'Facebook', 'price': '27', 'exchange': 'Nasdaq'}


## Add one column to the table

In [39]:
collection.update_many({},{"$set":{"volume":"10000"}})
print("After Adding volume column:")
for i in collection.find():
    print(i)

After Adding volume column:
{'_id': ObjectId('612da6aa261ab319fa8517c2'), 'id': '101', 'stock': 'Google', 'price': '26', 'exchange': 'London', 'volume': '10000'}
{'_id': ObjectId('612da6aa261ab319fa8517c3'), 'id': '102', 'stock': 'Facebook', 'price': '27', 'exchange': 'Nasdaq', 'volume': '10000'}


## Delete other collection in our Mongo Database 

In [37]:
print(db.list_collection_names())

['mongo_db', 'mongo_table', 'mongodb_table', 'system.indexes']


In [38]:
print("Collection in our database:")
print(db.list_collection_names())
print("\n")
example = db['mongo_db']
example.drop()
print("After dropping one of them:")
print(db.list_collection_names())

Collection in our database:
['mongo_db', 'mongo_table', 'mongodb_table', 'system.indexes']


After dropping one of them:
['mongo_table', 'mongodb_table', 'system.indexes']


### MySQL versus MongoDB:  
MySQL represents data in tables and rows, while MongoDB represents data as of JSON documents. Additionally, MySQL uses SQL as query language, while MongoDB uses Javascript. Hence, MySQL is better suited for structured data while MongoDB can handle unstructured data. 